<a href="https://colab.research.google.com/github/Tasfia-007/CRISiSLab-Earthquake-Data-Collection/blob/main/Gamma_association_with_phasenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install obspy seisbench pyproj seaborn
!pip install git+https://github.com/wayneweiqiang/GaMMA.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.44
    Uninstalling SQLAlchemy-2.0.44:
      Successfully uninstalled SQLAlchemy-2.0.44
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
google-adk 1.17.0 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


  Cloning https://github.com/wayneweiqiang/GaMMA.git to /tmp/pip-req-build-onb0yi1i
  Running command git clone --filter=blob:none --quiet https://github.com/wayneweiqiang/GaMMA.git /tmp/pip-req-build-onb0yi1i
  Resolved https://github.com/wayneweiqiang/GaMMA.git to commit f6b1ac7680f50bcc7d5d3928361ba02a7df0f523
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 97.1 MB/s eta 0:00:00
  Created wheel for GMMA: filename=GMMA-1.2.17-py3-none-any.whl size=32837 sha256=f493fd9760087d2efdb39223e2f008192f149fa013adc8dc954d23b3cfae1082
  Stored in directory: /tmp/pip-ephem-wheel-cache-bxioarmr/wheels/97/45/22/9a91feb083fef5b93b0c4c52beb49d057eb7ae3c7eea92a657
Successfully built GMMA
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


# Seismic Event Detection using Windowed GAMMA Association

## Overview
This code implements earthquake event detection from seismic phase picks using the GAMMA (Gaussian Mixture Model Association) algorithm. It processes PhaseNet-generated P-wave and S-wave picks through time-windowed analysis with velocity grid search optimization.

---

## Problem Statement & Initial Fix

### Critical Station ID Mismatch
The first major issue was that pick IDs didn't match station codes in the station dataframe. Pick IDs came in format `NETWORK.STATION.LOCATION.CHANNEL` (e.g., `NZ.WEL.10.HHZ`), while the station dataframe only had station codes.

**Solution:**
```python
# Extract middle component (station code) from pick IDs
pick_df['id'] = pick_df['id'].str.split('.').str[1]
```

This extraction achieved 100% match rate - critical because without matching station IDs, we can't retrieve station coordinates needed for earthquake location.

---

## Coordinate System Transformation

Since we're working with New Zealand data, I converted from WGS84 (lat/lon) to NZTM projection (EPSG:2193) - a local Cartesian coordinate system in kilometers.

```python
transformer = Transformer.from_crs(wgs84, local_crs)
station_df["x(km)"] = ...  # Easting coordinate
station_df["y(km)"] = ...  # Northing coordinate  
station_df["z(km)"] = -station_df["depth_km"]  # Negative because depth is downward
```

**Why this matters:** GAMMA requires distances in km for travel-time calculations. Lat/lon degrees don't work directly for distance calculations.

---

## Time Windowing Strategy

### Why 15-Minute Windows?
Processing the entire dataset at once is computationally prohibitive. The data spans ~180 hours, so I implemented a sliding window approach:

- **Window size:** 15 minutes
- **Minimum picks per window:** 2 (below this, association is meaningless)
- **Non-overlapping windows:** Sequential processing

**Window Creation Logic:**
```python
current_time = start_time
while current_time < end_time:
    window_end = current_time + timedelta(minutes=WINDOW_SIZE)
    mask = (pick_df['timestamp'] >= current_time) & (pick_df['timestamp'] < window_end)
    if mask.sum() >= MIN_PICKS:
        windows.append({...})
    current_time = window_end
```

This reduces memory usage and allows parallel processing (though not implemented here).

---

## Velocity Grid Search - The Core Innovation

### The Velocity Problem
Earthquake location accuracy critically depends on seismic wave velocities (Vp for P-waves, Vs for S-waves). But we don't know the exact velocity structure beforehand!

### Solution: Exhaustive Grid Search
multiple velocity models(ak-125f) and select the one that produces the most coherent events:

```python
vel_p_values = [1.45, 1.65, 5.8, 6.8, 8]      # km/s - P-wave velocities
vel_s_values = [1.0, 3.2, 3.9, 4.48, 4.49, 4.5]  # km/s - S-wave velocities
```

**Velocity ranges explained:**
- **1.45-1.65 km/s:** Shallow sedimentary layers, water-saturated zones
- **5.8-6.8 km/s (Vp):** Typical continental crust velocities
- **8.0 km/s (Vp):** Lower crust / upper mantle
- **3.2-4.5 km/s (Vs):** Corresponding S-wave velocities

**Physical constraint:** Always `Vs < Vp` (S-waves are inherently slower than P-waves)

### Grid Search Implementation
For each time window:
```python
for vp in vel_p_values:
    for vs in vel_s_values:
        if vs >= vp:  # Invalid combination - skip
            continue
        
        config["vel"] = {"p": vp, "s": vs}
        catalogs, assignments = association(window_picks, station_df, config)
        n_events = len(catalogs)
        
        # Keep the velocity model that detects the MOST events
        if n_events > best_n_events:
            best_n_events = n_events
            best_catalog = catalogs
            best_params = {'vp': vp, 'vs': vs}
```

**Rationale:** The velocity model that produces the maximum number of well-constrained events is most likely correct for that region and time period. This is an implicit optimization criterion.

---

## GAMMA Configuration Parameters

### Spatial Search Bounds
Extended 20 km beyond the station network:
```python
"x(km)": (x_min - 20, x_max + 20)
"y(km)": (y_min - 20, y_max + 20)  
"z(km)": (0, max(200, z_max + 20))
```
The 20 km buffer accounts for events outside the network that can still be detected by distant stations.

### DBSCAN Clustering Parameters
GAMMA uses a two-stage approach:
1. **BGMM (Bayesian Gaussian Mixture Model):** Generates candidate event locations
2. **DBSCAN:** Clusters picks in time-space to identify individual events

```python
"dbscan_eps": 10,              # seconds - temporal clustering threshold
"dbscan_min_samples": 2,       # minimum picks to form a cluster
"min_picks_per_eq": 2,         # minimum picks required per event
```

**Why these values:**
- `eps=10s`: Events separated by >10s are distinct
- `min_samples=2`: Conservative - need at least 2 picks to define an event
- This minimizes false positives at the cost of potentially missing small events

### Method Selection
```python
"method": "BGMM"              # Bayesian Gaussian Mixture Model
"oversample_factor": 4         # 4x density of candidate points
```
BGMM is preferred over standard GMM for sparse data because it handles varying cluster sizes better and has built-in regularization.

### Uncertainty Constraints
```python
"max_sigma11": 3.0,   # Maximum uncertainty in x-direction (km)
"max_sigma22": 1.5,   # Maximum uncertainty in y-direction (km)
"max_sigma12": 1.5,   # Maximum covariance between x-y
```
Events with uncertainties exceeding these thresholds are automatically filtered out. This ensures we only keep well-constrained locations.

### BFGS Optimization Bounds
```python
"bfgs_bounds": (
    (x_min - 1, x_max + 1),  # Allow slight extension beyond spatial bounds
    (y_min - 1, y_max + 1),
    (0, z_max + 1),           # Depth must be positive
    (None, None),             # Unconstrained origin time
)
```
GAMMA uses BFGS optimization to refine event locations. These bounds prevent the optimizer from wandering into physically unrealistic regions.

---

## Output Files

### 1. `catalog_windowed(15).csv`
Contains detected earthquake events with:
- **x, y, z:** Event location in NZTM coordinates (km)
- **time:** Origin time of the earthquake
- **sigma11, sigma22, sigma12:** Location uncertainties and covariances
- **gamma_score:** Association quality metric (0-1, higher is better)
- **window_idx, window_start:** Metadata about which time window detected this event

### 2. `assignments_windowed(15).csv`
Pick-to-event associations:
- **pick_index:** Index of the original pick in the input dataframe
- **event_index:** Which event this pick is assigned to
- **gamma_score:** Confidence of this specific association
- **window_idx:** Which time window produced this association

This file is crucial for quality control - you can trace back which picks contributed to each event.

### 3. `window_results(15).csv`
Summary statistics per time window:
- **n_events:** Number of events detected in this window
- **n_picks:** Total picks available in this window
- **vp, vs:** Best velocity model for this window
- **window_start:** Window timestamp

This provides insights into temporal variations in seismicity and velocity structure.

---


In [1]:
import pandas as pd
import numpy as np
from gamma.utils import association
from itertools import product
import json
from datetime import datetime, timedelta

# Load data
print("Loading picks and stations...")
pick_df = pd.read_csv("/kaggle/working/phasenet_picks.csv")
station_csv = "/kaggle/working/available_waveform_summary.csv"
station_df = pd.read_csv(station_csv)

# ============================================================
# FIX: Extract station code from pick IDs
# ============================================================
print("\n" + "="*60)
print("FIXING STATION ID MISMATCH")
print("="*60)

print(f"\nBefore fix:")
print(f"  Sample pick IDs: {pick_df['id'].iloc[:3].tolist()}")
print(f"  Sample station IDs: {station_df['station_code'].iloc[:3].tolist()}")

# Extract station code (middle part between dots)
pick_df['id'] = pick_df['id'].str.split('.').str[1]

print(f"\nAfter fix:")
print(f"  Sample pick IDs: {pick_df['id'].iloc[:3].tolist()}")

# Verify matches
matches = pick_df['id'].isin(station_df['station_code']).sum()
print(f"\n✓ Matching stations: {matches}/{len(pick_df)} picks ({100*matches/len(pick_df):.1f}%)")

if matches == 0:
    print("\n⚠ ERROR: Still no matches! Check station_code column name")
    print(f"Station dataframe columns: {station_df.columns.tolist()}")
    exit()

# ============================================================
# Setup coordinate transformation
# ============================================================
from pyproj import CRS, Transformer
wgs84 = CRS.from_epsg(4326)
local_crs = CRS.from_epsg(2193) #for nz
transformer = Transformer.from_crs(wgs84, local_crs)

station_df["x(km)"] = station_df.apply(
    lambda row: transformer.transform(row["station_latitude"], row["station_longitude"])[0] / 1000, axis=1
)
station_df["y(km)"] = station_df.apply(
    lambda row: transformer.transform(row["station_latitude"], row["station_longitude"])[1] / 1000, axis=1
)
station_df["z(km)"] = -station_df["depth_km"]
station_df["id"] = station_df["station_code"]
station_df = station_df.drop_duplicates(subset=["id"]).reset_index(drop=True)

# Set bounding box
x_min, x_max = station_df["x(km)"].min(), station_df["x(km)"].max()
y_min, y_max = station_df["y(km)"].min(), station_df["y(km)"].max()
z_min, z_max = station_df["z(km)"].min(), station_df["z(km)"].max()

print(f"\nData Summary:")
print(f"Total picks: {len(pick_df)}")
print(f"P picks: {sum(pick_df['type'] == 'p')}")
print(f"S picks: {sum(pick_df['type'] == 's')}")
print(f"Unique stations: {len(station_df)}")
print(f"X range: ({x_min:.1f}, {x_max:.1f}) km")
print(f"Y range: ({y_min:.1f}, {y_max:.1f}) km")
print(f"Z range: ({z_min:.1f}, {z_max:.1f}) km")

# Convert timestamps
pick_df['timestamp'] = pd.to_datetime(pick_df['timestamp'])
time_span = (pick_df['timestamp'].max() - pick_df['timestamp'].min()).total_seconds()
print(f"Time span of picks: {time_span:.1f} seconds ({time_span/3600:.1f} hours)")

# ============================================================
# 15-MINUTE WINDOW PROCESSING WITH VELOCITY GRID SEARCH
# ============================================================
print("\n" + "="*60)
print("PROCESSING IN 30-MINUTE WINDOWS")
print("="*60)

# Fixed parameters
WINDOW_SIZE = 15 # minutes   1500*180 merged divide 15 minutes
MIN_PICKS = 2
DBSCAN_EPS = 10
DBSCAN_MIN_SAMPLES = 2
MIN_PICKS_PER_EQ = 2

# Velocity grid (reduced for efficiency)
vel_p_values = [1.45, 1.65, 5.8, 6.8, 8]
vel_s_values = [1.0, 3.2, 3.9, 4.48, 4.49, 4.5]

print(f"\nFixed parameters:")
print(f"  Window size: {WINDOW_SIZE} minutes")
print(f"  Min picks per window: {MIN_PICKS}")
print(f"  DBSCAN eps: {DBSCAN_EPS} seconds")
print(f"  DBSCAN min_samples: {DBSCAN_MIN_SAMPLES}")
print(f"  Min picks per event: {MIN_PICKS_PER_EQ}")

print(f"\nVelocity grid:")
print(f"  Vp values: {vel_p_values}")
print(f"  Vs values: {vel_s_values}")
print(f"  Total combinations: {len(vel_p_values) * len(vel_s_values)}")

# Create time windows
start_time = pick_df['timestamp'].min()
end_time = pick_df['timestamp'].max()
current_time = start_time

windows = []
while current_time < end_time:
    window_end = current_time + timedelta(minutes=WINDOW_SIZE)
    mask = (pick_df['timestamp'] >= current_time) & (pick_df['timestamp'] < window_end)
    n_picks = mask.sum()
    if n_picks >= MIN_PICKS:
        windows.append({
            'start': current_time,
            'end': window_end,
            'n_picks': n_picks
        })
    current_time = window_end

print(f"\nTotal windows: {len(windows)}")
print(f"Windows with ≥{MIN_PICKS} picks: {len(windows)}")

# Base configuration
base_config = {
    "dims": ['x(km)', 'y(km)', 'z(km)'],
    "use_dbscan": True,
    "use_amplitude": False,
    "x(km)": (x_min - 20, x_max + 20),
    "y(km)": (y_min - 20, y_max + 20),
    "z(km)": (0, max(200, z_max + 20)),
    "method": "BGMM",
    "oversample_factor": 4,
    "dbscan_eps": DBSCAN_EPS,
    "dbscan_min_samples": DBSCAN_MIN_SAMPLES,
    "min_picks_per_eq": MIN_PICKS_PER_EQ,
    "max_sigma11": 3.0,
    "max_sigma22": 1.5,
    "max_sigma12": 1.5,
}

base_config["bfgs_bounds"] = (
    (base_config["x(km)"][0] - 1, base_config["x(km)"][1] + 1),
    (base_config["y(km)"][0] - 1, base_config["y(km)"][1] + 1),
    (0, base_config["z(km)"][1] + 1),
    (None, None),
)

# Process windows with velocity grid search
all_catalogs = []
all_assignments = []
results = []

start_processing = datetime.now()

for win_idx, window in enumerate(windows):
    print(f"\n{'='*60}")
    print(f"Window {win_idx+1}/{len(windows)}: {window['start']} ({window['n_picks']} picks)")
    print(f"{'='*60}")

    # Extract picks for this window
    mask = (pick_df['timestamp'] >= window['start']) & (pick_df['timestamp'] < window['end'])
    window_picks = pick_df[mask].reset_index(drop=True)

    best_n_events = 0
    best_catalog = None
    best_assignments = None
    best_params = None

    # Grid search over velocities
    for vp in vel_p_values:
        for vs in vel_s_values:
            if vs >= vp:  # Skip invalid Vs >= Vp
                continue

            config = base_config.copy()
            config["vel"] = {"p": vp, "s": vs}

            try:
                catalogs, assignments = association(window_picks, station_df, config, method=config["method"])
                n_events = len(catalogs)

                if n_events > best_n_events:
                    best_n_events = n_events
                    best_catalog = catalogs
                    best_assignments = assignments
                    best_params = {'vp': vp, 'vs': vs}

            except Exception as e:
                continue

    # Save best result for this window
    if best_n_events > 0:
        catalog_df = pd.DataFrame(best_catalog)
        catalog_df['window_idx'] = win_idx
        catalog_df['window_start'] = window['start']

        assignments_df = pd.DataFrame(best_assignments, columns=["pick_index", "event_index", "gamma_score"])
        assignments_df['window_idx'] = win_idx

        all_catalogs.append(catalog_df)
        all_assignments.append(assignments_df)

        avg_picks = assignments_df.groupby('event_index').size().mean()
        print(f"\n✓ Found {best_n_events} events (Vp={best_params['vp']}, Vs={best_params['vs']:.1f})")
        print(f"  Avg picks per event: {avg_picks:.1f}")

        results.append({
            'window_idx': win_idx,
            'window_start': window['start'],
            'n_events': best_n_events,
            'n_picks': window['n_picks'],
            'vp': best_params['vp'],
            'vs': best_params['vs']
        })
    else:
        print(f"\n✗ No events found in this window")

# ============================================================
# COMBINE AND SAVE RESULTS
# ============================================================
print("\n" + "="*60)
print("COMBINING RESULTS")
print("="*60)

if all_catalogs:
    final_catalog = pd.concat(all_catalogs, ignore_index=True)
    final_assignments = pd.concat(all_assignments, ignore_index=True)
    results_df = pd.DataFrame(results)

    final_catalog.to_csv("/kaggle/working/catalog_windowed(15).csv", index=False)
    final_assignments.to_csv("/kaggle/working/assignments_windowed(15).csv", index=False)
    results_df.to_csv("/kaggle/working/window_results(15).csv", index=False)

    print(f"\n✓ Total events detected: {len(final_catalog)}")
    print(f"✓ Total assignments: {len(final_assignments)}")
    print(f"✓ Windows with events: {len(results_df)}/{len(windows)}")
    print(f"\n✓ Catalog saved to: catalog_windowed.csv")
    print(f"✓ Assignments saved to: assignments_windowed.csv")
    print(f"✓ Window summary saved to: window_results.csv")

    # Summary statistics
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"\nTotal events: {len(final_catalog)}")
    print(f"Events per window (avg): {results_df['n_events'].mean():.1f}")
    print(f"Most common Vp: {results_df['vp'].mode().values[0]} km/s")
    print(f"Most common Vs: {results_df['vs'].mode().values[0]:.1f} km/s")

else:
    print("\n⚠ No events detected in any window!")

elapsed = (datetime.now() - start_processing).total_seconds()
print(f"\n⏱ Total processing time: {elapsed/60:.1f} minutes")

ModuleNotFoundError: No module named 'gamma'

# Earthquake Event Analysis & Classification

## Overview
This script analyzes the detected earthquake catalog from GAMMA and classifies events based on quality, spatial clustering, and temporal patterns.

---

## 1. Quality-Based Classification

Events are categorized into 4 quality levels based on number of picks and gamma score:

```python
HIGH_QUALITY    = (num_picks >= 10) & (gamma_score >= 15)
MEDIUM_QUALITY  = (num_picks >= 5) & (gamma_score >= 5)
LOW_QUALITY     = (num_picks >= 3)
NOISE           = (num_picks < 3)
```

**Rationale:**
- More picks = better location constraint
- Higher gamma score = stronger association confidence
- Events with <3 picks are likely false detections

---

## 2. Spatial Clustering (DBSCAN)

Groups events that are spatially close to identify seismic zones:

```python
DBSCAN(eps=20, min_samples=2)
```

**Parameters:**
- `eps=20 km`: Events within 20km radius are considered neighbors
- `min_samples=2`: At least 2 events needed to form a cluster
- Isolated events labeled as `-1` (noise)

**Output:** Identifies active seismic zones and isolated events

---

## 3. Temporal Clustering

Identifies sequences of events happening close in time:

```python
temporal_sequence = (time_diff > 30 minutes).cumsum()
```

**Logic:**
- Calculates time gap between consecutive events
- Gaps >30 minutes define new sequences
- Useful for identifying aftershock sequences or swarms

---

## 4. Statistics by Quality

Computes average metrics for each quality category:
- Number of picks (P + S waves)
- Gamma association score
- Depth and depth range
- Spatial distribution

---

## Output Files

### `catalog_classified.csv`
Enhanced catalog with additional columns:
- `quality`: HIGH/MEDIUM/LOW/NOISE
- `spatial_cluster`: Cluster ID or -1 for isolated
- `temporal_sequence`: Sequence ID
- `time_diff_minutes`: Gap since previous event

---

## Visualization (9-Panel Plot)

### Row 1: Spatial Analysis
1. **Event Map by Quality**: Color-coded by quality (red=high, gray=noise)
2. **Depth Distribution**: Histogram of event depths per quality level
3. **Spatial Clusters**: Map showing DBSCAN clusters

### Row 2: Event Characteristics
4. **Pick Distribution**: Histogram of number of picks per event
5. **Gamma Score Distribution**: Quality metric distribution
6. **Event Timeline**: Events per day over time

### Row 3: Cross-Sections & Relationships
7. **X-Z Cross-Section**: Vertical slice showing depth profile
8. **P vs S Picks**: Scatter plot (colored by gamma score)
9. **Quality Pie Chart**: Percentage breakdown of quality levels

---

## 3D Visualization

Additional 3D plots showing:
- **Events by Quality**: 3D spatial distribution colored by quality
- **Spatial Clusters**: 3D view of DBSCAN clusters
- **Events by Metrics**: Size = number of picks, Color = gamma score

---

## Key Insights from Analysis

### Quality Distribution
Shows what percentage of detections are reliable:
- High quality → Well-constrained, trustworthy locations
- Noise → Should be filtered out before further analysis

### Spatial Clustering
Reveals:
- Active fault zones (large clusters)
- Isolated events (possible triggered events)
- Spatial extent of seismicity

### Temporal Patterns
Identifies:
- Mainshock-aftershock sequences
- Earthquake swarms (sustained activity)
- Background seismicity vs. sequences

---

## Usage

```python
# Load and run analysis
catalog = pd.read_csv("catalog_windowed(60).csv")
# ... run classification and clustering ...

# Filter high-quality events only
high_quality = catalog[catalog['quality'] == 'HIGH']

# Analyze specific cluster
cluster_0 = catalog[catalog['spatial_cluster'] == 0]
```

---



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from datetime import datetime
import seaborn as sns

# Load the catalog
catalog = pd.read_csv("/kaggle/working/catalog_windowed(60).csv")

print("="*60)
print("EARTHQUAKE EVENT ANALYSIS")
print("="*60)

# Basic statistics
print(f"\nTotal detected events: {len(catalog)}")
print(f"Unique windows: {catalog['window_idx'].nunique()}")
print(f"Date range: {catalog['time'].min()} to {catalog['time'].max()}")

# ============================================================
# 1. QUALITY-BASED CLASSIFICATION
# ============================================================
print("\n" + "="*60)
print("EVENT QUALITY CLASSIFICATION")
print("="*60)

# Define quality thresholds
HIGH_QUALITY = (catalog['num_picks'] >= 10) & (catalog['gamma_score'] >= 15)
MEDIUM_QUALITY = (catalog['num_picks'] >= 5) & (catalog['gamma_score'] >= 5) & (~HIGH_QUALITY)
LOW_QUALITY = (catalog['num_picks'] >= 3) & (~HIGH_QUALITY) & (~MEDIUM_QUALITY)
NOISE = catalog['num_picks'] < 3

catalog['quality'] = 'NOISE'
catalog.loc[LOW_QUALITY, 'quality'] = 'LOW'
catalog.loc[MEDIUM_QUALITY, 'quality'] = 'MEDIUM'
catalog.loc[HIGH_QUALITY, 'quality'] = 'HIGH'

print("\nQuality Distribution:")
quality_counts = catalog['quality'].value_counts()
for quality, count in quality_counts.items():
    pct = 100 * count / len(catalog)
    print(f"  {quality:8s}: {count:3d} events ({pct:5.1f}%)")

# ============================================================
# 2. SPATIAL CLUSTERING
# ============================================================
print("\n" + "="*60)
print("SPATIAL CLUSTERING ANALYSIS")
print("="*60)

# Extract coordinates
coords = catalog[['x(km)', 'y(km)', 'z(km)']].values

# DBSCAN clustering (eps in km)
dbscan = DBSCAN(eps=20, min_samples=2)  # 20km radius
catalog['spatial_cluster'] = dbscan.fit_predict(coords)

n_clusters = len(set(catalog['spatial_cluster'])) - (1 if -1 in catalog['spatial_cluster'] else 0)
n_noise = (catalog['spatial_cluster'] == -1).sum()

print(f"\nSpatial clusters found: {n_clusters}")
print(f"Isolated events (noise): {n_noise}")
print(f"Clustered events: {len(catalog) - n_noise}")

# Cluster statistics
if n_clusters > 0:
    print("\nTop 5 Largest Clusters:")
    cluster_sizes = catalog[catalog['spatial_cluster'] != -1]['spatial_cluster'].value_counts()
    for i, (cluster_id, size) in enumerate(cluster_sizes.head(5).items(), 1):
        cluster_events = catalog[catalog['spatial_cluster'] == cluster_id]
        center_x = cluster_events['x(km)'].mean()
        center_y = cluster_events['y(km)'].mean()
        center_z = cluster_events['z(km)'].mean()
        print(f"  Cluster {cluster_id}: {size} events at ({center_x:.1f}, {center_y:.1f}, {center_z:.1f}) km")

# ============================================================
# 3. TEMPORAL CLUSTERING
# ============================================================
print("\n" + "="*60)
print("TEMPORAL CLUSTERING ANALYSIS")
print("="*60)

catalog['time'] = pd.to_datetime(catalog['time'])
catalog = catalog.sort_values('time').reset_index(drop=True)

# Calculate time differences (in minutes)
time_diffs = catalog['time'].diff().dt.total_seconds() / 60
catalog['time_diff_minutes'] = time_diffs

# Find temporal sequences (events within 30 minutes)
catalog['temporal_sequence'] = (time_diffs > 30).cumsum()

n_sequences = catalog['temporal_sequence'].nunique()
print(f"\nTemporal sequences (>30 min gaps): {n_sequences}")

sequence_sizes = catalog['temporal_sequence'].value_counts().head(5)
print("\nTop 5 Most Active Sequences:")
for seq_id, size in sequence_sizes.items():
    seq_events = catalog[catalog['temporal_sequence'] == seq_id]
    duration = (seq_events['time'].max() - seq_events['time'].min()).total_seconds() / 60
    print(f"  Sequence {seq_id}: {size} events over {duration:.1f} minutes")

# ============================================================
# 4. STATISTICS BY QUALITY
# ============================================================
print("\n" + "="*60)
print("STATISTICS BY QUALITY")
print("="*60)

for quality in ['HIGH', 'MEDIUM', 'LOW', 'NOISE']:
    subset = catalog[catalog['quality'] == quality]
    if len(subset) > 0:
        print(f"\n{quality} QUALITY EVENTS ({len(subset)}):")
        print(f"  Avg picks: {subset['num_picks'].mean():.1f}")
        print(f"  Avg gamma score: {subset['gamma_score'].mean():.1f}")
        print(f"  Avg depth: {subset['z(km)'].mean():.1f} km")
        print(f"  Depth range: {subset['z(km)'].min():.1f} - {subset['z(km)'].max():.1f} km")

# ============================================================
# SAVE CLASSIFIED DATA
# ============================================================
catalog.to_csv("/kaggle/working/catalog_classified.csv", index=False)
print("\n✓ Classified catalog saved to: catalog_classified.csv")

# ============================================================
# PLOTTING
# ============================================================
print("\n" + "="*60)
print("GENERATING PLOTS")
print("="*60)

fig = plt.figure(figsize=(20, 14))

# Plot 1: Map view with quality colors
ax1 = plt.subplot(3, 3, 1)
colors = {'HIGH': 'red', 'MEDIUM': 'orange', 'LOW': 'yellow', 'NOISE': 'gray'}
for quality in ['NOISE', 'LOW', 'MEDIUM', 'HIGH']:
    subset = catalog[catalog['quality'] == quality]
    ax1.scatter(subset['x(km)'], subset['y(km)'],
               c=colors[quality], s=50, alpha=0.6, label=quality, edgecolors='black', linewidth=0.5)
ax1.set_xlabel('X (km)', fontsize=10)
ax1.set_ylabel('Y (km)', fontsize=10)
ax1.set_title('Event Map by Quality', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Depth distribution by quality
ax2 = plt.subplot(3, 3, 2)
for quality in ['HIGH', 'MEDIUM', 'LOW', 'NOISE']:
    subset = catalog[catalog['quality'] == quality]
    if len(subset) > 0:
        ax2.hist(subset['z(km)'], bins=20, alpha=0.5, label=quality, color=colors[quality])
ax2.set_xlabel('Depth (km)', fontsize=10)
ax2.set_ylabel('Count', fontsize=10)
ax2.set_title('Depth Distribution by Quality', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Spatial clusters
ax3 = plt.subplot(3, 3, 3)
unique_clusters = catalog[catalog['spatial_cluster'] != -1]['spatial_cluster'].unique()
cmap = plt.cm.get_cmap('tab20', len(unique_clusters))
for i, cluster_id in enumerate(unique_clusters):
    subset = catalog[catalog['spatial_cluster'] == cluster_id]
    ax3.scatter(subset['x(km)'], subset['y(km)'],
               c=[cmap(i)], s=100, alpha=0.7, label=f'C{cluster_id}', edgecolors='black', linewidth=0.5)
# Plot noise
noise = catalog[catalog['spatial_cluster'] == -1]
ax3.scatter(noise['x(km)'], noise['y(km)'],
           c='lightgray', s=30, alpha=0.5, label='Isolated', edgecolors='black', linewidth=0.3)
ax3.set_xlabel('X (km)', fontsize=10)
ax3.set_ylabel('Y (km)', fontsize=10)
ax3.set_title(f'Spatial Clusters (n={n_clusters})', fontsize=12, fontweight='bold')
ax3.legend(ncol=2, fontsize=8)
ax3.grid(True, alpha=0.3)

# Plot 4: Number of picks distribution
ax4 = plt.subplot(3, 3, 4)
ax4.hist(catalog['num_picks'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax4.axvline(catalog['num_picks'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {catalog["num_picks"].mean():.1f}')
ax4.set_xlabel('Number of Picks', fontsize=10)
ax4.set_ylabel('Count', fontsize=10)
ax4.set_title('Pick Distribution', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Gamma score distribution
ax5 = plt.subplot(3, 3, 5)
ax5.hist(catalog['gamma_score'], bins=30, color='darkgreen', alpha=0.7, edgecolor='black')
ax5.axvline(catalog['gamma_score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {catalog["gamma_score"].mean():.1f}')
ax5.set_xlabel('Gamma Score', fontsize=10)
ax5.set_ylabel('Count', fontsize=10)
ax5.set_title('Gamma Score Distribution', fontsize=12, fontweight='bold')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Plot 6: Timeline
ax6 = plt.subplot(3, 3, 6)
catalog['date'] = catalog['time'].dt.date
events_per_day = catalog.groupby('date').size()
ax6.plot(events_per_day.index, events_per_day.values, marker='o', linewidth=2, markersize=6, color='purple')
ax6.set_xlabel('Date', fontsize=10)
ax6.set_ylabel('Events per Day', fontsize=10)
ax6.set_title('Event Timeline', fontsize=12, fontweight='bold')
ax6.tick_params(axis='x', rotation=45)
ax6.grid(True, alpha=0.3)

# Plot 7: Cross-section (X-Z)
ax7 = plt.subplot(3, 3, 7)
for quality in ['NOISE', 'LOW', 'MEDIUM', 'HIGH']:
    subset = catalog[catalog['quality'] == quality]
    ax7.scatter(subset['x(km)'], subset['z(km)'],
               c=colors[quality], s=50, alpha=0.6, label=quality, edgecolors='black', linewidth=0.5)
ax7.set_xlabel('X (km)', fontsize=10)
ax7.set_ylabel('Depth (km)', fontsize=10)
ax7.set_title('Cross-Section (X-Z)', fontsize=12, fontweight='bold')
ax7.invert_yaxis()
ax7.legend()
ax7.grid(True, alpha=0.3)

# Plot 8: P vs S picks
ax8 = plt.subplot(3, 3, 8)
ax8.scatter(catalog['num_p_picks'], catalog['num_s_picks'],
           c=catalog['gamma_score'], s=80, alpha=0.6, cmap='viridis', edgecolors='black', linewidth=0.5)
ax8.plot([0, catalog['num_p_picks'].max()], [0, catalog['num_p_picks'].max()],
         'r--', linewidth=2, label='P=S line')
ax8.set_xlabel('P Picks', fontsize=10)
ax8.set_ylabel('S Picks', fontsize=10)
ax8.set_title('P vs S Picks (color=gamma score)', fontsize=12, fontweight='bold')
ax8.legend()
ax8.grid(True, alpha=0.3)
cbar = plt.colorbar(ax8.collections[0], ax=ax8)
cbar.set_label('Gamma Score', fontsize=9)

# Plot 9: Quality pie chart
ax9 = plt.subplot(3, 3, 9)
quality_counts = catalog['quality'].value_counts()
ax9.pie(quality_counts.values, labels=quality_counts.index, autopct='%1.1f%%',
        colors=[colors[q] for q in quality_counts.index], startangle=90)
ax9.set_title('Event Quality Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('/kaggle/working/earthquake_analysis.png', dpi=150, bbox_inches='tight')
print("✓ Plot saved to: earthquake_analysis.png")

# ============================================================
# ADDITIONAL DETAILED PLOTS
# ============================================================

# 3D plot if mpl_toolkits available
try:
    from mpl_toolkits.mplot3d import Axes3D

    fig2 = plt.figure(figsize=(15, 5))

    # 3D plot by quality
    ax = fig2.add_subplot(131, projection='3d')
    for quality in ['NOISE', 'LOW', 'MEDIUM', 'HIGH']:
        subset = catalog[catalog['quality'] == quality]
        ax.scatter(subset['x(km)'], subset['y(km)'], subset['z(km)'],
                  c=colors[quality], s=50, alpha=0.6, label=quality)
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Depth (km)')
    ax.set_title('3D Event Distribution by Quality')
    ax.legend()
    ax.invert_zaxis()

    # 3D plot by cluster
    ax2 = fig2.add_subplot(132, projection='3d')
    for i, cluster_id in enumerate(unique_clusters[:10]):  # Top 10 clusters
        subset = catalog[catalog['spatial_cluster'] == cluster_id]
        ax2.scatter(subset['x(km)'], subset['y(km)'], subset['z(km)'],
                   s=100, alpha=0.7, label=f'C{cluster_id}')
    noise = catalog[catalog['spatial_cluster'] == -1]
    ax2.scatter(noise['x(km)'], noise['y(km)'], noise['z(km)'],
               c='lightgray', s=20, alpha=0.3, label='Isolated')
    ax2.set_xlabel('X (km)')
    ax2.set_ylabel('Y (km)')
    ax2.set_zlabel('Depth (km)')
    ax2.set_title('3D Spatial Clusters')
    ax2.legend(ncol=2)
    ax2.invert_zaxis()

    # 3D plot sized by gamma score
    ax3 = fig2.add_subplot(133, projection='3d')
    scatter = ax3.scatter(catalog['x(km)'], catalog['y(km)'], catalog['z(km)'],
                         c=catalog['gamma_score'], s=catalog['num_picks']*10,
                         alpha=0.6, cmap='plasma')
    ax3.set_xlabel('X (km)')
    ax3.set_ylabel('Y (km)')
    ax3.set_zlabel('Depth (km)')
    ax3.set_title('Events (size=picks, color=gamma)')
    ax3.invert_zaxis()
    plt.colorbar(scatter, ax=ax3, label='Gamma Score')

    plt.tight_layout()
    plt.savefig('/kaggle/working/earthquake_3d_analysis.png', dpi=150, bbox_inches='tight')
    print("✓ 3D plot saved to: earthquake_3d_analysis.png")
except:
    print("⚠ 3D plotting not available")

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)
print(f"\nSummary:")
print(f"  Total events: {len(catalog)}")
print(f"  High quality: {(catalog['quality'] == 'HIGH').sum()}")
print(f"  Medium quality: {(catalog['quality'] == 'MEDIUM').sum()}")
print(f"  Low quality: {(catalog['quality'] == 'LOW').sum()}")
print(f"  Noise: {(catalog['quality'] == 'NOISE').sum()}")
print(f"  Spatial clusters: {n_clusters}")
print(f"  Temporal sequences: {n_sequences}")